# Simulación en Python usando Simpy: llegadas y servicio

---
Realizaremos una simulación en Python usando Simpy de eventos discretos (simulación de llegadas y servicio). 


El ejemplo que vamos a utilizar es el siguiente: Una peluquería tiene un peluquero que se demora entre 15 y 30 minutos por corte. La peluquería recibe en promedio 3 clientes por hora (es decir, uno cada 20 minutos). Se desea simular las llegadas y servicios de 5 clientes. 

Una vez realizado el código, podremos variar cualquiera de éstos parámetros (tener varios peluqueros, cambiar el tiempo que se demoran por corte, simular para n clientes o para un tiempo específico como una cierta cantidad de minutos, etc).


## Simulación de eventos discretos

La peluquería es atendida por un solo peluquero y un corte de pelo se lleva entre 15 y 30 minutos, distribuidos de una manera uniforme. 

**_Tiempos de llegadas_**: El tiempo entre llegadas de los clientes a la peluquería es exponencial con media de λ=20 minutos. Los clientes son atendidos con base en la disciplina FIFO (primero en llegar, primero en salir), de modo que las llegadas son calculadas de acuerdo a la siguiente fórmula:

T_LLEGADAS = –λ ln(R)

donde R es un número pseudoaleatorio.


**_Tiempos de servicio_**: Los tiempos de servicio son calculados de acuerdo a la siguiente fórmula:

tiempo_corte = TIEMPO_CORTE_MIN + (TIEMPO_CORTE_MAX – TIEMPO_CORTE_MIN)*R

Esto es: el mínimo de tiempo que se demora el peluquero, en nuestro ejemplo es 15, más la diferencia entre el máximo y el mínimo, en nuestro ejemplo serían 15 minutos (30 menos 15), multiplicado por un número pseudoaleatorio. El resultado nos dará un número entre 15 y 30.

Solucionemos este ejercicio usando los siguientes comandos de Simpy:
env_...     : Para hacer tal chimbada
yoield



In [0]:
# EJECUTAR PARA INSTALAR simpy
!pip install simpy

In [0]:
# Cargando las bibliotecas a utilizar
import random   # Para generar números pseudoaleatorios
import math     # Para la funcion logaritmo
import simpy    # Proporciona un entorno de simulacion


# Definoendo las constantes y variables a utilizar
SEMILLA = 30
NUM_PELUQUEROS = 1
TIEMPO_CORTE_MIN = 15
TIEMPO_CORTE_MAX = 30
T_LLEGADAS = 20
TIEMPO_SIMULACION = 480  # min
#TOT_CLIENTES = 5
 
te  = 0.0   # tiempo de espera total
dt  = 0.0   # duracion de servicio total
fin = 0.0   # minuto en el que finaliza


# Procedimientos
def cortar(cliente):
	global dt                                           # Para poder acceder a la variable dt declarada anteriormente
	R = random.random()                                 # Obtiene un numero aleatorio y lo guarda en R
	tiempo = TIEMPO_CORTE_MAX - TIEMPO_CORTE_MIN  
	tiempo_corte = TIEMPO_CORTE_MIN + (tiempo*R)        # Distribucion uniforme
	yield env.timeout(tiempo_corte)                     # deja correr el tiempo n minutos
	print(" \o/ Corte listo a %s en %.2f minutos" % (cliente,tiempo_corte))
	dt = dt + tiempo_corte                              # Acumula los tiempos de uso de la i


def cliente (env, name, personal ):
	global te
	global fin
	llega = env.now                                     # Guarda el minuto de llegada del cliente
	print ("---> %s llego a peluqueria en minuto %.2f" % (name, llega))
	with personal.request() as request:                 # Espera su turno
		yield request                                     # Obtiene turno
		pasa = env.now                                    # Guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega                             # Calcula el tiempo que espero
		te = te + espera                                  # Acumula los tiempos de espera
		print ("**** %s pasa con peluquero en minuto %.2f habiendo esperado %.2f" % (name, pasa, espera))
		yield env.process(cortar(name))                   # Invoca al proceso cortar
		deja = env.now                                    # Guarda el minuto en que termina el proceso cortar 
		print ("<--- %s deja peluqueria en minuto %.2f" % (name, deja))
		fin = deja                                        # Conserva globalmente el ultimo minuto de la simulacion
	

def principal (env, personal):
	llegada = 0
	i = 0
	while True:                       
		R = random.random()
		llegada = -T_LLEGADAS * math.log(R)               # Distribucion exponencial
		yield env.timeout(llegada)                        # Deja transcurrir un tiempo entre uno y otro
		i += 1
		env.process(cliente(env, 'Cliente %d' % i, personal))
    

# PROGRAMA PRINCIPAL    
print ("------------------- Bienvenido Simulacion Peluqueria ------------------")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
personal = simpy.Resource(env, NUM_PELUQUEROS) #Crea los recursos (peluqueros)
env.process(principal(env, personal)) #Invoca el proceso princial
env.run(until = TIEMPO_SIMULACION) #Inicia la simulacion

# Calculo de medidas de desempeño
print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_PELUQUEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")



------------------- Bienvenido Simulacion Peluqueria ------------------
---> Cliente 1 llego a peluqueria en minuto 12.36
**** Cliente 1 pasa con peluquero en minuto 12.36 habiendo esperado 0.00
 \o/ Corte listo a Cliente 1 en 15.45 minutos
<--- Cliente 1 deja peluqueria en minuto 27.81
---> Cliente 2 llego a peluqueria en minuto 37.17
**** Cliente 2 pasa con peluquero en minuto 37.17 habiendo esperado 0.00
---> Cliente 3 llego a peluqueria en minuto 45.67
 \o/ Corte listo a Cliente 2 en 18.15 minutos
<--- Cliente 2 deja peluqueria en minuto 55.32
**** Cliente 3 pasa con peluquero en minuto 55.32 habiendo esperado 9.65
---> Cliente 4 llego a peluqueria en minuto 72.83
 \o/ Corte listo a Cliente 3 en 20.96 minutos
<--- Cliente 3 deja peluqueria en minuto 76.28
**** Cliente 4 pasa con peluquero en minuto 76.28 habiendo esperado 3.45
---> Cliente 5 llego a peluqueria en minuto 81.70
---> Cliente 6 llego a peluqueria en minuto 97.17
---> Cliente 7 llego a peluqueria en minuto 97.30
---> Cl

# CONCLUSIÓN

La presente simulación en Python usando Simpy es una herramienta para obtener resultados en forma más sencilla que si lo hiciéramos manualmente. Es tambien muy flexible y se pueden crear programas más elaborados que si utilizáramos una hoja de cálculo.

Python es un lenguaje multiplataforma, ademas, es software libre, por lo que es ideal para cualquier proyecto académico.

Es posible adaptar este programa para resolver diferentes problemas y se puede ampliar para obtener más datos cuya solución manual o matemática pudiera ser complicada.
